https://www.wired.com/story/best-wordle-tips/

In [ ]:
%load_ext kedro.ipython

In [ ]:
%reload_kedro

In [3]:
import sys
sys.path.append("../../src/projectwordle")

In [4]:
import polars as pl
import numpy as np
from IPython.display import HTML
from projectwordle.utils import (
    color_pattern_matching,
    difficulty_distribution,
)

pl.Config(tbl_rows=50)

# Load Data

In [ ]:
five_letter_word_anagrams = catalog.load("five_letter_words_anagrams")
alternative_openers = catalog.load("alternative_openers")

In [6]:
alternative_openers.head(6)

index,challenge,guess,match_pattern,letter_differences,common_letters,num_diff_letters,num_common_letters,num_matching_index,num_choices_after_guess,possible_guesses,challenge_in_possible_guesses,next_guess,group,tries,difficulty,guess_word_anagrams,anagram_num
u32,str,str,str,str,str,u8,u8,u8,u16,str,bool,str,u32,u8,str,str,u8
0,"""yummy""","""soare""","""BBBBB""","""oesar""","""""",5,0,0,538,"""digit, limit, thing, fifth, th…",false,"""linty""",0,null,"""very hard""","""soare""",1
1,"""yummy""","""linty""","""BBBBG""","""oteislnar""","""y""",9,1,1,50,"""puppy, duchy, muddy, buddy, ju…",false,"""puppy""",0,null,"""very hard""","""linty""",1
2,"""yummy""","""puppy""","""BGBBG""","""oteislnarp""","""yu""",10,2,2,34,"""fuzzy, dummy, duchy, buddy, bu…",true,"""fuzzy""",0,null,"""very hard""","""puppy""",1
3,"""yummy""","""fuzzy""","""BGBBG""","""otezislnarpf""","""yu""",12,2,2,24,"""muddy, buggy, duchy, mummy, bu…",true,"""muddy""",0,null,"""very hard""","""fuzzy""",1
4,"""yummy""","""muddy""","""YGBBG""","""otdezislnarpf""","""yum""",13,3,2,3,"""gummy, yummy, jumby""",true,"""gummy""",0,null,"""very hard""","""muddy""",1
5,"""yummy""","""gummy""","""BGGGG""","""gotdezislnarpf""","""yum""",14,3,3,1,"""yummy""",true,"""yummy""",0,null,"""very hard""","""gummy""",1


In [7]:
alternative_openers.height

54528

In [8]:
simulating_tries_difficulty = (
    alternative_openers
    .select(
        "challenge", "guess", "letter_differences", "common_letters",
        "num_diff_letters", "num_common_letters", "num_matching_index",
        "match_pattern", "num_choices_after_guess", "possible_guesses",
        "challenge_in_possible_guesses", "next_guess", "group", "tries",
        "difficulty", "guess_word_anagrams", "anagram_num"
    )
    .join(
        five_letter_word_anagrams.select("words", "anagrams", "anagram_num"),
        left_on="next_guess",
        right_on="words",
        how="left",
        coalesce=True
    )
)

In [9]:
simulating_tries_difficulty.head()

challenge,guess,letter_differences,common_letters,num_diff_letters,num_common_letters,num_matching_index,match_pattern,num_choices_after_guess,possible_guesses,challenge_in_possible_guesses,next_guess,group,tries,difficulty,guess_word_anagrams,anagram_num,anagrams,anagram_num_right
str,str,str,str,u8,u8,u8,str,u16,str,bool,str,u32,u8,str,str,u8,str,u8
"""yummy""","""soare""","""oesar""","""""",5,0,0,"""BBBBB""",538,"""digit, limit, thing, fifth, th…",false,"""linty""",0,null,"""very hard""","""soare""",1,"""linty""",1
"""yummy""","""linty""","""oteislnar""","""y""",9,1,1,"""BBBBG""",50,"""puppy, duchy, muddy, buddy, ju…",false,"""puppy""",0,null,"""very hard""","""linty""",1,"""puppy""",1
"""yummy""","""puppy""","""oteislnarp""","""yu""",10,2,2,"""BGBBG""",34,"""fuzzy, dummy, duchy, buddy, bu…",true,"""fuzzy""",0,null,"""very hard""","""puppy""",1,"""fuzzy""",1
"""yummy""","""fuzzy""","""otezislnarpf""","""yu""",12,2,2,"""BGBBG""",24,"""muddy, buggy, duchy, mummy, bu…",true,"""muddy""",0,null,"""very hard""","""fuzzy""",1,"""muddy""",1
"""yummy""","""muddy""","""otdezislnarpf""","""yum""",13,3,2,"""YGBBG""",3,"""gummy, yummy, jumby""",true,"""gummy""",0,null,"""very hard""","""muddy""",1,"""gummy""",1


In [10]:
(
    simulating_tries_difficulty
    .unique(subset=["group"], keep="first")
    ["difficulty"]
    .value_counts()
    .sort("count", descending=True)
)

difficulty,count
str,u32
"""moderate""",5394
"""hard""",3067
"""very hard""",625
"""easy""",2


# Create dataframe of first guesses

In [11]:
# Group by "Category" and return the first row of each group
first_guess = (
    simulating_tries_difficulty
    .group_by("group", maintain_order=True)
    .first()
)

first_guess.head()

group,challenge,guess,letter_differences,common_letters,num_diff_letters,num_common_letters,num_matching_index,match_pattern,num_choices_after_guess,possible_guesses,challenge_in_possible_guesses,next_guess,tries,difficulty,guess_word_anagrams,anagram_num,anagrams,anagram_num_right
u32,str,str,str,str,u8,u8,u8,str,u16,str,bool,str,u8,str,str,u8,str,u8
0,"""yummy""","""soare""","""oesar""","""""",5,0,0,"""BBBBB""",538,"""digit, limit, thing, fifth, th…",false,"""linty""",null,"""very hard""","""soare""",1,"""linty""",1
1,"""calyx""","""soare""","""roes""","""a""",4,1,0,"""BBYBB""",716,"""catch, japan, amply, human, fa…",false,"""linty""",4,"""moderate""","""soare""",1,"""linty""",1
2,"""varas""","""soare""","""eo""","""sar""",2,3,0,"""YBYYB""",75,"""paris, argus, arias, bursa, ma…",false,"""linty""",5,"""hard""","""soare""",1,"""linty""",1
3,"""bubba""","""soare""","""roes""","""a""",4,1,0,"""BBYBB""",716,"""human, apply, badly, adult, fa…",false,"""linty""",5,"""hard""","""soare""",1,"""linty""",1
4,"""lowes""","""soare""","""ar""","""soe""",2,3,1,"""YGBBY""",27,"""jones, moses, nosed, loess, eo…",true,"""linty""",4,"""moderate""","""soare""",1,"""linty""",1


In [12]:
avg_tries = round(first_guess["tries"].mean(), 2)
avg_tries

4.23

# EDA

In [13]:
# Plot difficulty distribution
difficulty_distribution(
    simulating_tries_difficulty
)

## Most frequent number of guesses

In [14]:
(
    first_guess
    ["tries"]
    .value_counts()
    .sort(by="count", descending=True)
)

tries,count
u8,u32
4,3350
5,2143
3,2044
6,924
null,625
2,1
1,1


### Challenge words without `r` , `s` or `t`
These are some of the most frequently ocuuring letters. We can then see how our naive algorithm operates in coming to the correct guess.

In [15]:
challenge_words_without_r_s_t = (
    first_guess
    .filter(
        ~(pl.col("challenge").str.contains("r|s|t"))
        & ((pl.col("challenge") != (pl.col("guess"))))
    )
)

challenge_words_without_r_s_t.sample(n = 20, with_replacement = False)

group,challenge,guess,letter_differences,common_letters,num_diff_letters,num_common_letters,num_matching_index,match_pattern,num_choices_after_guess,possible_guesses,challenge_in_possible_guesses,next_guess,tries,difficulty,guess_word_anagrams,anagram_num,anagrams,anagram_num_right
u32,str,str,str,str,u8,u8,u8,str,u16,str,bool,str,u8,str,str,u8,str,u8
1174,"""villa""","""soare""","""roes""","""a""",4,1,0,"""BBYBB""",716,"""faith, human, china, cajun, wa…",false,"""linty""",3,"""moderate""","""soare""",1,"""linty""",1
5252,"""imido""","""soare""","""ares""","""o""",4,1,0,"""BYBBB""",237,"""blood, union, ditto, cloth, cl…",false,"""linty""",5,"""hard""","""soare""",1,"""linty""",1
3040,"""volva""","""soare""","""res""","""oa""",3,2,1,"""BGYBB""",118,"""local, woman, total, loyal, to…",false,"""linty""",4,"""moderate""","""soare""",1,"""linty""",1
6310,"""bible""","""soare""","""aros""","""e""",4,1,1,"""BBBBG""",284,"""judge, twice, title, bible, pi…",true,"""linty""",3,"""moderate""","""soare""",1,"""linty""",1
34,"""aweel""","""soare""","""ros""","""ae""",3,2,0,"""BBYBY""",234,"""legal, amend, ideal, metal, ad…",false,"""linty""",6,"""hard""","""soare""",1,"""linty""",1
2825,"""boxen""","""soare""","""ars""","""oe""",3,2,1,"""BGBBY""",86,"""money, novel, model, hotel, to…",false,"""linty""",null,"""very hard""","""soare""",1,"""linty""",1
7299,"""bunny""","""soare""","""oesar""","""""",5,0,0,"""BBBBB""",538,"""billy, input, night, limit, th…",false,"""linty""",4,"""moderate""","""soare""",1,"""linty""",1
2896,"""gluon""","""soare""","""ares""","""o""",4,1,0,"""BYBBB""",237,"""knock, blood, photo, clock, bl…",false,"""linty""",4,"""moderate""","""soare""",1,"""linty""",1
1202,"""lindy""","""soare""","""oesar""","""""",5,0,0,"""BBBBB""",538,"""think, unity, unify, child, li…",false,"""linty""",3,"""moderate""","""soare""",1,"""linty""",1


In [16]:
# Plot difficulty distribution of of challenge words that doesn't contain 'r', 's' or 't'
difficulty_distribution(challenge_words_without_r_s_t)

## Null values (Incomplete games)

In [17]:
incomplete_games = (
    simulating_tries_difficulty
    .filter(pl.col("tries").is_null())
)

In [18]:
incomplete_games.head()

challenge,guess,letter_differences,common_letters,num_diff_letters,num_common_letters,num_matching_index,match_pattern,num_choices_after_guess,possible_guesses,challenge_in_possible_guesses,next_guess,group,tries,difficulty,guess_word_anagrams,anagram_num,anagrams,anagram_num_right
str,str,str,str,u8,u8,u8,str,u16,str,bool,str,u32,u8,str,str,u8,str,u8
"""yummy""","""soare""","""oesar""","""""",5,0,0,"""BBBBB""",538,"""digit, limit, thing, fifth, th…",false,"""linty""",0,null,"""very hard""","""soare""",1,"""linty""",1
"""yummy""","""linty""","""oteislnar""","""y""",9,1,1,"""BBBBG""",50,"""puppy, duchy, muddy, buddy, ju…",false,"""puppy""",0,null,"""very hard""","""linty""",1,"""puppy""",1
"""yummy""","""puppy""","""oteislnarp""","""yu""",10,2,2,"""BGBBG""",34,"""fuzzy, dummy, duchy, buddy, bu…",true,"""fuzzy""",0,null,"""very hard""","""puppy""",1,"""fuzzy""",1
"""yummy""","""fuzzy""","""otezislnarpf""","""yu""",12,2,2,"""BGBBG""",24,"""muddy, buggy, duchy, mummy, bu…",true,"""muddy""",0,null,"""very hard""","""fuzzy""",1,"""muddy""",1
"""yummy""","""muddy""","""otdezislnarpf""","""yum""",13,3,2,"""YGBBG""",3,"""gummy, yummy, jumby""",true,"""gummy""",0,null,"""very hard""","""muddy""",1,"""gummy""",1


In [19]:
null_select_group = np.random.choice(incomplete_games["group"], 1, replace = False)[0]

HTML(
    incomplete_games
    .filter(pl.col("group") == null_select_group)
    .to_pandas()
    .assign(
        match_pattern = lambda df_:
        color_pattern_matching(
            dataf=df_,
            challenge_col="challenge",
            guess_col="guess"
        )
    )
    [[
        "challenge", "guess", "match_pattern", "next_guess",
        "num_choices_after_guess", "possible_guesses", "difficulty",
    ]]
    .to_html(escape=False)
)

,challenge,guess,match_pattern,next_guess,num_choices_after_guess,possible_guesses,difficulty
0,jammy,soare,soare,linty,716,"human, watch, faint, daily, faith, iliad, laugh, habit, apply, paint",very hard
1,jammy,linty,linty,happy,50,"happy, daddy, caddy, paddy, baggy, gawky, gaudy, gabby, cabby, jacky",very hard
2,jammy,happy,happy,bawdy,34,"bawdy, gaudy, daddy, baggy, mammy, cabby, jacky, gabby, gawky, wacky",very hard
3,jammy,bawdy,bawdy,mammy,12,"mammy, jacky, gauzy, jazzy, faggy, gaumy, gaucy, cacky, gamay, jammy",very hard
4,jammy,mammy,mammy,gaumy,3,"gaumy, jammy, gammy",very hard
5,jammy,gaumy,gaumy,jammy,1,jammy,very hard


In [20]:
incomplete_games_check = (
    incomplete_games
    .filter(pl.col("group") == null_select_group)
    .select(
        "challenge", "guess", "match_pattern", "num_choices_after_guess",
        "possible_guesses", "challenge_in_possible_guesses", "next_guess"
    )
)

incomplete_games_check

challenge,guess,match_pattern,num_choices_after_guess,possible_guesses,challenge_in_possible_guesses,next_guess
str,str,str,u16,str,bool,str
"""jammy""","""soare""","""BBYBB""",716,"""human, watch, faint, daily, fa…",false,"""linty"""
"""jammy""","""linty""","""BBBBG""",50,"""happy, daddy, caddy, paddy, ba…",false,"""happy"""
"""jammy""","""happy""","""BGBBG""",34,"""bawdy, gaudy, daddy, baggy, ma…",false,"""bawdy"""
"""jammy""","""bawdy""","""BGBBG""",12,"""mammy, jacky, gauzy, jazzy, fa…",true,"""mammy"""
"""jammy""","""mammy""","""YGGGG""",3,"""gaumy, jammy, gammy""",true,"""gaumy"""
"""jammy""","""gaumy""","""BGBGG""",1,"""jammy""",true,"""jammy"""


In [21]:
(
    incomplete_games_check
    .gather_every(n=1)
    .head(1)
    ["possible_guesses"]
    .item()
)

'human, watch, faint, daily, faith, iliad, laugh, habit, apply, paint'

In [22]:
(
    incomplete_games_check
    .gather_every(n=1, offset=1)
    .head(1)
    ["possible_guesses"]
    .item()
)

'happy, daddy, caddy, paddy, baggy, gawky, gaudy, gabby, cabby, jacky'

In [23]:
(
    incomplete_games_check
    .gather_every(n=1, offset=2)
    .head(1)
    ["possible_guesses"]
    .item()
)

'bawdy, gaudy, daddy, baggy, mammy, cabby, jacky, gabby, gawky, wacky'

In [24]:
(
    incomplete_games_check
    .gather_every(n=1, offset=3)
    .head(1)
    ["possible_guesses"]
    .item()
)

'mammy, jacky, gauzy, jazzy, faggy, gaumy, gaucy, cacky, gamay, jammy'

In [25]:
(
    incomplete_games_check
    .gather_every(n=1, offset=4)
    .head(1)
    ["possible_guesses"]
    .item()
)

'gaumy, jammy, gammy'

In [26]:
(
    incomplete_games_check
    .gather_every(n=1, offset=5)
    .head(1)
    ["possible_guesses"]
    .item()
)

'jammy'

# Challenge word statistics

In [27]:
challenge_stats = (
    simulating_tries_difficulty
    .group_by("challenge")
    .agg(
        tries_mode=pl.col("tries").mode().cast(pl.UInt8),
        tries_mean=(pl.col("tries").mean().cast(pl.Float32)).round(3),
        tries_null_pct=(pl.col("tries").is_null().mean().cast(pl.Float32) * 100).round(3),
    )
    .explode("tries_mode")
    .sort("tries_null_pct", descending = True)
)

In [28]:
challenge_stats.head()

challenge,tries_mode,tries_mean,tries_null_pct
str,u8,f32,f32
"""japer""",null,null,100.0
"""gauzy""",null,null,100.0
"""jujus""",null,null,100.0
"""hepar""",null,null,100.0
"""doody""",null,null,100.0


In [29]:
print(f"Overall incompletion rate: {(first_guess['tries'].is_null().sum() / first_guess.height) * 100:.2f}%")

Overall incompletion rate: 6.88%


## Retrieve the 2nd row of every group of 6 rows.

### Determine the average number of choices remaining after the second guess. This will tell us how effective the two guess strategy is at solving the challenge word.

In [30]:
second_guess_df = (
    simulating_tries_difficulty
    .with_row_index(name="row_num")
    .filter(
        (pl.col("row_num") % 6 == 1) &
        ~(pl.col("guess") == "saint") # remove instance where 'saint' as first guess is correct
    )
    .drop("row_num")
)

In [31]:
second_guess_df.head()

challenge,guess,letter_differences,common_letters,num_diff_letters,num_common_letters,num_matching_index,match_pattern,num_choices_after_guess,possible_guesses,challenge_in_possible_guesses,next_guess,group,tries,difficulty,guess_word_anagrams,anagram_num,anagrams,anagram_num_right
str,str,str,str,u8,u8,u8,str,u16,str,bool,str,u32,u8,str,str,u8,str,u8
"""yummy""","""linty""","""oteislnar""","""y""",9,1,1,"""BBBBG""",50,"""puppy, duchy, muddy, buddy, ju…",false,"""puppy""",0,null,"""very hard""","""linty""",1,"""puppy""",1
"""calyx""","""linty""","""oteisnr""","""aly""",7,3,0,"""YBBBY""",11,"""alkyl, calyx, phyla, allyl, by…",true,"""alkyl""",1,4,"""moderate""","""linty""",1,"""alkyl""",1
"""varas""","""linty""","""oteilny""","""sar""",7,3,0,"""BBBBB""",32,"""marsh, harsh, bursa, rajas, au…",false,"""marsh""",2,5,"""hard""","""linty""",1,"""marsh""",1
"""bubba""","""linty""","""yoteislnr""","""a""",9,1,0,"""BBBBB""",59,"""madam, mamma, gamma, magma, wa…",false,"""madam""",3,5,"""hard""","""linty""",1,"""madam""",1
"""lowes""","""linty""","""yatinr""","""soel""",6,4,2,"""GBBBB""",4,"""loess, lowes, losel, lomes""",true,"""loess""",4,4,"""moderate""","""linty""",1,"""loess""",1


In [32]:
# More efficiently 
second_guess_df_2 = (
    simulating_tries_difficulty
    .gather_every(n=6, offset=1)
    .filter(~(pl.col("guess") == "saint")) # remove instance where 'saint' as first guess is correct
)

second_guess_df_2.head()

challenge,guess,letter_differences,common_letters,num_diff_letters,num_common_letters,num_matching_index,match_pattern,num_choices_after_guess,possible_guesses,challenge_in_possible_guesses,next_guess,group,tries,difficulty,guess_word_anagrams,anagram_num,anagrams,anagram_num_right
str,str,str,str,u8,u8,u8,str,u16,str,bool,str,u32,u8,str,str,u8,str,u8
"""yummy""","""linty""","""oteislnar""","""y""",9,1,1,"""BBBBG""",50,"""puppy, duchy, muddy, buddy, ju…",false,"""puppy""",0,null,"""very hard""","""linty""",1,"""puppy""",1
"""calyx""","""linty""","""oteisnr""","""aly""",7,3,0,"""YBBBY""",11,"""alkyl, calyx, phyla, allyl, by…",true,"""alkyl""",1,4,"""moderate""","""linty""",1,"""alkyl""",1
"""varas""","""linty""","""oteilny""","""sar""",7,3,0,"""BBBBB""",32,"""marsh, harsh, bursa, rajas, au…",false,"""marsh""",2,5,"""hard""","""linty""",1,"""marsh""",1
"""bubba""","""linty""","""yoteislnr""","""a""",9,1,0,"""BBBBB""",59,"""madam, mamma, gamma, magma, wa…",false,"""madam""",3,5,"""hard""","""linty""",1,"""madam""",1
"""lowes""","""linty""","""yatinr""","""soel""",6,4,2,"""GBBBB""",4,"""loess, lowes, losel, lomes""",true,"""loess""",4,4,"""moderate""","""linty""",1,"""loess""",1


In [33]:
print(f'Avg num choices after 2nd guess: {round(second_guess_df_2["num_choices_after_guess"].mean(), 2)}')

Avg num choices after 2nd guess: 16.22


### Greatest num of choices left after second guess

In [34]:
max_num_choices_after_2nd_guess = (
    second_guess_df_2
    .sort("num_choices_after_guess", descending=True)
    ["num_choices_after_guess"]
    .head(1)
    .item()
)

max_num_choices_after_2nd_guess

101

In [35]:
(
    second_guess_df_2
    .filter(pl.col("num_choices_after_guess") == max_num_choices_after_2nd_guess)
    ["challenge"]
    .unique()
    .to_numpy()
)


array(['hazer', 'regma', 'barer', 'rehab', 'dread', 'pareu', 'arepa',
       'pawer', 'varec', 'paper', 'abear', 'warez', 'aread', 'wafer',
       'agger', 'dazer', 'gazer', 'auger', 'raver', 'cream', 'amber',
       'remap', 'pacer', 'rewax', 'recap', 'adder', 'gaper', 'fader',
       'arear', 'waver', 'macer', 'rebar', 'waker', 'dewar', 'debar',
       'raper', 'japer', 'begar', 'raker', 'pager', 'harem', 'caver',
       'caber', 'freak', 'paver', 'feuar', 'wader', 'eager', 'wager',
       'rarer', 'razer', 'creak', 'acker', 'wreak', 'drear', 'herma',
       'daker', 'parer', 'regar', 'gamer', 'hepar', 'jager', 'carex',
       'break', 'kerma', 'dream', 'maker', 'areca', 'caper', 'armer',
       'haver', 'zerda', 'perea', 'mazer', 'carer', 'gager', 'embar',
       'bread', 'faver', 'rager', 'gerah', 'parev', 'afear', 'derma',
       'baker', 'kebar', 'aredd', 'cedar', 'farer', 'racer', 'facer',
       'armed', 'bream', 'cager', 'ardeb', 'rawer', 'eggar', 'faker',
       'waxer', 'am

### How often on average is the challenge word in the possible choices after the second guess

In [36]:
print(f'Avg times challenge in possible guesses: {round(second_guess_df_2["challenge_in_possible_guesses"].mean() * 100, 2)}%')

Avg times challenge in possible guesses: 74.79%


In [37]:
(
    simulating_tries_difficulty
    .filter(pl.col("challenge") == "hound")
)

challenge,guess,letter_differences,common_letters,num_diff_letters,num_common_letters,num_matching_index,match_pattern,num_choices_after_guess,possible_guesses,challenge_in_possible_guesses,next_guess,group,tries,difficulty,guess_word_anagrams,anagram_num,anagrams,anagram_num_right
str,str,str,str,u8,u8,u8,str,u16,str,bool,str,u32,u8,str,str,u8,str,u8
"""hound""","""soare""","""ares""","""o""",4,1,1,"""BGBBB""",340,"""tommy, young, youth, dough, mo…",false,"""linty""",2248,6,"""hard""","""soare""",1,"""linty""",1
"""hound""","""linty""","""yteislar""","""on""",8,2,1,"""BBYBB""",12,"""wound, pound, mound, hound, co…",true,"""wound""",2248,6,"""hard""","""linty""",1,"""wound""",1
"""hound""","""wound""","""ywteislar""","""onud""",9,4,4,"""BGGGG""",3,"""pound, mound, hound""",true,"""pound""",2248,6,"""hard""","""wound""",1,"""pound""",1
"""hound""","""pound""","""ywteislarp""","""onud""",10,4,4,"""BGGGG""",2,"""mound, hound""",true,"""mound""",2248,6,"""hard""","""pound""",1,"""mound""",1
"""hound""","""mound""","""ywtmeislarp""","""onud""",11,4,4,"""BGGGG""",1,"""hound""",true,"""hound""",2248,6,"""hard""","""mound""",1,"""hound""",1
"""hound""","""hound""","""""","""onudh""",0,5,5,"""GGGGG""",0,"""""",false,"""hound""",2248,6,"""hard""","""hound""",1,"""hound""",1


In [38]:
HTML(
    simulating_tries_difficulty
    .filter(pl.col("group") == 2248)
    .to_pandas()
    .assign(
        match_pattern = lambda df_:
        color_pattern_matching(
            dataf=df_,
            challenge_col="challenge",
            guess_col="guess"
        )
    )
    [[
        "challenge", "guess", "match_pattern", "next_guess",
        "num_choices_after_guess", "possible_guesses", "difficulty",
    ]]
    .to_html(escape=False)
)


,challenge,guess,match_pattern,next_guess,num_choices_after_guess,possible_guesses,difficulty
0,hound,soare,soare,linty,340,"tommy, young, youth, dough, mouth, month, point, doubt, joint, toxic",hard
1,hound,linty,linty,wound,12,"wound, pound, mound, hound, codon, novum, jomon, nohow, boong, cogon",hard
2,hound,wound,wound,pound,3,"pound, mound, hound",hard
3,hound,pound,pound,mound,2,"mound, hound",hard
4,hound,mound,mound,hound,1,hound,hard
5,hound,hound,hound,hound,0,,hard


In [39]:
(
    simulating_tries_difficulty
    .filter(pl.col("challenge") == "night")
)

challenge,guess,letter_differences,common_letters,num_diff_letters,num_common_letters,num_matching_index,match_pattern,num_choices_after_guess,possible_guesses,challenge_in_possible_guesses,next_guess,group,tries,difficulty,guess_word_anagrams,anagram_num,anagrams,anagram_num_right
str,str,str,str,u8,u8,u8,str,u16,str,bool,str,u32,u8,str,str,u8,str,u8
"""night""","""soare""","""oesar""","""""",5,0,0,"""BBBBB""",538,"""light, click, think, child, fi…",false,"""linty""",5828,3,"""moderate""","""soare""",1,"""linty""",1
"""night""","""linty""","""yoeslar""","""int""",7,3,1,"""BGYYB""",3,"""night, nicht, nitid""",true,"""night""",5828,3,"""moderate""","""linty""",1,"""night, thing""",2
"""night""","""night""","""""","""intgh""",0,5,5,"""GGGGG""",0,"""""",false,"""night""",5828,3,"""moderate""","""night, thing""",2,"""night, thing""",2
"""night""","""night""","""""","""intgh""",0,5,5,"""GGGGG""",0,"""""",false,"""night""",5828,3,"""moderate""","""night, thing""",2,"""night, thing""",2
"""night""","""night""","""""","""intgh""",0,5,5,"""GGGGG""",0,"""""",false,"""night""",5828,3,"""moderate""","""night, thing""",2,"""night, thing""",2
"""night""","""night""","""""","""intgh""",0,5,5,"""GGGGG""",0,"""""",false,"""night""",5828,3,"""moderate""","""night, thing""",2,"""night, thing""",2


In [40]:
HTML(
    simulating_tries_difficulty
    .filter(pl.col("group") == 5828)
    .to_pandas()
    .assign(
        match_pattern = lambda df_:
        color_pattern_matching(
            dataf=df_,
            challenge_col="challenge",
            guess_col="guess"
        )
    )
    [[
        "challenge", "guess", "match_pattern", "next_guess",
        "num_choices_after_guess", "possible_guesses", "difficulty",
    ]]
    .to_html(escape=False)
)


,challenge,guess,match_pattern,next_guess,num_choices_after_guess,possible_guesses,difficulty
0,night,soare,soare,linty,538,"light, click, think, child, fifth, willy, thing, civil, pupil, lucid",moderate
1,night,linty,linty,night,3,"night, nicht, nitid",moderate
2,night,night,night,night,0,,moderate
3,night,night,night,night,0,,moderate
4,night,night,night,night,0,,moderate
5,night,night,night,night,0,,moderate
